# Closures

- recap of scope
- we hav global scope and local scope
- scope affects  our variable

In [2]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [3]:
b = 6
f1(3)

3
6


In [12]:
b = 2
def f2(a):
    print(a)
    print('b',b)
    b = 2
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

When Python compiles the body of the function, it decides that *b* is a local variabe because it is assigned within the function.

Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is **local**.

If we want the interpreter to treat *b* here as **global** variable, we use the global declaration:

In [22]:
b = 'hello'
b = 2
def f2(a):
    global b   #global declaration
    print(a)
    b = 'adw'
    print('b',b)
    print(locals())
    b = 'afdwa'
    print(b)
f2(3)
print(b)

3
b adw
{'a': 3}
afdwa
afdwa


We have seen two scopes in action:

*The module global scope* Made of names assigned to values outside of any class or function block.

*The f2 function local scope* Made of names assigned to values as parameters, or directly in the body of the function.

## Closures

A closure is a function *f* with an extended scope that includes variables referenced in the body of f that are not global or local variables of *f*.
Iz can access nonglobal variables outside the function *f* body.

In [40]:

def make_averager():
    series = []
    series2 = []
    def averager(new_value):
        print(series2)
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    def t2():
        pass
    return averager

In [41]:

avg = make_averager()
avg(10)

[]


10.0

In [42]:
avg(11)

[]


10.5

In [43]:
avg(12)

[]


11.0

Where does the *avg* function find the series?

Note that **series** is a local variable of *make_averager* because the assignment **series** =[] happens in the body of the function.
But when *avg(10)* is called, *make_averager* has already returned, and its local scope is lon gone.

Within *averager*, **series** is a *free variable*. 


![closure](free_var.png)

We can look at the variables of our function by looking at the __ code__ attributes. __ code__ represents the compiled body of the function.


In [44]:
avg.__code__.co_varnames

('new_value', 'total')

In [45]:
avg.__code__.co_freevars

('series', 'series2')

In [46]:
avg.__closure__

(<cell at 0x7f57a09398b0: list object at 0x7f57a08b2b80>,
 <cell at 0x7f57a0939e80: list object at 0x7f57a083f580>)

In [47]:
avg.__closure__[1].cell_contents

[]

In [39]:
avg.__closure__

IndexError: tuple index out of range

A closure is a function that keeps the binding of the free variable that exists when the function is defined, so that they can be used later when the function is called and the defining scope is no longer available.

*lexical scope*: The body of a function is evaluated in the environment where the function is defined, not the environment where the function is called. 
The scope where a object was defined we also call *lexical scope*.

In [75]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count = count + 1
        total = total + new_value
        return total / count
    return averager
    
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [76]:
avg.__code__.co_varnames

('new_value', 'count', 'total')

In [77]:
avg.__code__.co_freevars

()

In [78]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count = count + 1
        total = total + new_value
        return total / count
    return averager
    
avg = make_averager()
avg(10)

10.0

In [79]:
avg.__code__.co_varnames

('new_value',)

In [80]:
avg.__code__.co_freevars

('count', 'total')

*count* and *total* are immutable and all you can do is read, never update. If you try to rebind them, as in *count= count + 1*, then you are implicitly creating a local variable *count*. Therefore, it's not longer a free variable, but a local variable and will be not saved in the closure.

To declare variables as free variables us the **nonlocal** keyword.

# Exercise


In [81]:
def triple():
    a = 3
    def multiply(b):
        return a * b
    return multiply
triple()(6)


18

what is the free variable and the local variable?
what is the closure?
use _.co_varnames, __ code__.co_freevars, and __ closure__[0].cell_contents

In [3]:
def double():
    a = 2
    def multiply(b):
        return a * b
    return multiply

db = double()
db(2)

4

In [4]:
db(3)

6

In [9]:
def deco(func):
    def inner(a,b):
        return 3 * func(a,b)
    return inner

def multiply(a,b):
    return a * b

decorated = deco(multiply)
decorated.__code__.co_freevars
decorated(1,2)

6

In [10]:
decorated(2,2)

12

# Decorators

- let us mark functions
- enhance behavior of function

A decorator is a function that takes another function as argument.
A decorator may perform some processing with the decorated Function and returns it or replace it with another function object.

In [11]:
def decorate(func):
    func()
    def inner():
        print('runnning inner()')
    return inner

In [12]:
def target():
    print('running target()')
    
target = decorate(target)

running target()


In [13]:
target()

runnning inner()


In [14]:
@decorate
def target():
    print('running target()')

running target()


In [15]:
target()

runnning inner()


Strictly speaking decorators are just syntactic sugar.
- decorators are functions
- they usually define an inner function and return it to replace the decorated function. Code that uses inner functions almost always depends on closures

## Implementing a Simple Decorator

In [41]:
import time

def clock(fun):
    def clocked():      ####inner function
        t0 = time.time()
        result = fun()  ### fun is the free variable of the closure
        elapsed = time.time() - t0
        print("{elapsed}s func --> {result}".format(elapsed=elapsed, result=result))
        return result
    return clocked # return the inner function to replace the decorated function

In [42]:
def snooze():
    return 'Hello'

In [43]:
snooze = clock(snooze)

In [44]:
snooze()

9.5367431640625e-07s func --> Hello


'Hello'

In [45]:
@clock
def snooze():
    return 'Hello'

In [46]:
snooze()

2.86102294921875e-06s func --> Hello


'Hello'

In [47]:
def snooze(s):
    time.sleep(s)

In [48]:
snooze = clock(snooze)

In [49]:
snooze(1)

TypeError: clocked() takes 0 positional arguments but 1 was given

In [54]:
import time

def clock(fun):
    def clocked(n):      ####inner function
        t0 = time.time()
        result = fun(n)  # the parameter n is passed further down to fun, which is the
        # decorated function or target function
        elapsed = time.time() - t0
        print("{elapsed}s func --> {result}".format(elapsed=elapsed, result=result))
        return result
    return clocked # return the inner function to replace the decorated 

In [51]:
def snooze(s):  ### snooze is my clocked function which takes n
    time.sleep(s)

In [52]:
snooze = clock(snooze) 

In [53]:
snooze(1)

1.0010242462158203s func --> None


In [55]:
@clock
def snooze(s):
    time.sleep(s)

In [56]:
snooze(5)

5.004662275314331s func --> None


Faculty: $n!= 1 * 2 * .... * (n-1) * n$

In [66]:
@clock
def fac(n):
    result = 1
    if n < 1:
        return 1
    while n >= 1:
        result = result * n
        n = n - 1
    return result

fac(15)

2.6226043701171875e-06s func --> 1307674368000


1307674368000

In [70]:
def fac(n):
    result = 1
    if n < 1:
        return 1
    while n >= 1:
        result = result * n
        n = n - 1
    return result

fac = clock(fac)
fac(15)

4.291534423828125e-06s func --> 1307674368000


1307674368000

- the decorator replaces the decorated function with a new function (inner)
- it accepts the same argument and
- (usually) returns whatever the decorated funbction was suppose to return
- does extra processing
- attaches additional responsibilities dynamically